In [23]:
import glob; import sys; import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import ode
from matplotlib.lines import Line2D
from random import choices

pd.options.mode.chained_assignment = None
plt.rcParams.update({'font.size': 17}) # Set a good font size


rsol2AU = 0.00465047 # solar radii to au

In [3]:
def find_dir():
        '''
        Finds the likely location for the petar data files to be stored
        and gives the option to autoselect them.

        Returns data directory as a string
        '''

        # Finding possible directories where data could be stored
        directories = glob.glob('COMPAS_Output*')

        # Create a dictionary to store the available directories and index vals
        directoryList = {str(i): directories[i] for i in range(len(directories))}

        # Print the available directories
        print('Possible Directories:\n')
        for key, val in directoryList.items():
                print(key, ':', val)

        # Asking what directory the data is stored in and giving a list of potential directories
        chooseDirectory = input('\nWhat directory is the data stored in?  ')
        if chooseDirectory in directoryList.keys():
                dataDirectory = directoryList[str(chooseDirectory)]

        elif os.path.exists(str(chooseDirectory)):
                dataDirectory = str(chooseDirectory)

        else:
                print('Could not find directory\n')
                print('Quitting')
                sys.exit()

        return dataDirectory
    
def tdelay(ai,ei,m1,m2):
    """
    Calculates the GW timescale for a given binary
    semi-major axis, eccentricty and masses

    Input >>> ai = Semi Major axis [Rsol]
          ei = eccentricity
          m1, m2 = primary/secondary mass [Msol]

    Output >>> tGW = merger timescale [yrs]
    """


    # Defining useful constants
    Rsol = 6.9*(10**8.) #in meters
    MyrsToSec = 3.15*(10**13.) #time in sec
    tobs = 13*(10**3.)*MyrsToSec #Age of MilkyWay

    Gsi =  6.6*10**-11. #garavitaional constant in SI
    c = 3.*(10**8.) #velocity of light in seconds
    AUtoRsol = 214.9 #AU to Rsol
    Msol = 2.*(10**30) #Solar mass in kg
    betaWithoutMass = (64./5.)*(Gsi**3.0)/(c**5.0)
    daysToSeconds = 86400
    GyrsToSec = MyrsToSec * 1000
    YrsToSec = MyrsToSec/10**6


    #----tdelay

    #-- Choose ODE integrator
    backend = 'dopri5'

    l=len(ei)
    t_merger=[]

    for i in range(l):
        a0 = ai[i]*Rsol
        m_1 = m1[i]*Msol
        m_2 = m2[i]*Msol
        e0=ei[i]

        # If the initial ecc=0 then we have analytical solution
        if e0==0:
            beta = betaWithoutMass*m_1*m_2*(m_1+m_2)
            Te = (a0**4)/(4*beta)
            t_merger.append(Te/YrsToSec)
            continue

        c0Part1 = a0*(1. - e0**2.0)
        c0Part2 = (1.+(121./304.)*e0**2.)**(870./2299.)
        c0Part3 = e0**(12./19.)
        c0 = c0Part1/(c0Part2*c0Part3)
        beta = betaWithoutMass*m_1*m_2*(m_1+m_2)

        constant = (12./19.)*(c0**4.)/beta
        #print ((1. - e0**2.)**(3./2.))

        func = lambda e: constant*((e**(29./19.))*(1. + (121./304.)*e**2.)**(1181./2299.))/((1. - e**2.)**(3./2.))

        #-- Create ODE solver object
        solver = ode(func).set_integrator(backend)

        #-- Define initial and final parameters
        T0 = 0        #-- Initial value of T
        efinal = 1e-5 #-- Maximum value of e to integrate to

        solver.set_initial_value(T0, e0) #.set_f_params(r)

        sol = [] #-- Create an empty list to store the output in (here it will be the e list)

        #-- Define a function to append the output to our list
        def solout(e, T):
            sol.append([e, T/YrsToSec])
        solver.set_solout(solout)

        #-- This line actually integrates the ODE, no loop is required
        solver.integrate(efinal)

        #-- Convert list to array
        sol = np.asarray(sol, dtype=float)

        #-- Use sol to find the location

        e = sol[:, 0]
        T = np.abs(sol[:,1])

        t_max = max(np.abs(sol[:,1]))

        tm = t_max
        #print tm

        t_merger.append(tm)

    return np.asarray(t_merger)

def calcTrh(M, rh):
    '''
    Calculate the half-mass relaxation timescale for the cluster (Myrs)
    
    Input >>> M = cluster mass (Msol)
          >>> rh = cluster half-mass radius (pc)
    
    Output >>> trh = relaxation time (Myrs)
    '''
    
    #Define G 
    G = 0.00449830997959438 # pc^3 Msol^-1 Myrs^-2
    
    const = 0.138/(50*0.809) # Msol^-1
    
    return const * np.sqrt((M*rh**3)/(G))

def calcTint(M1, M2, a, Mcl, rh, trh):
    '''
    Calculates the interaction timescale for a hard encounter
    
    Input >>> M1 = mass 1 (Msol)
          >>> M2 = mass 2 (Msol)
          >>> a = semimajor axis (AU)
          >>> Mcl = cluster Mass (Msol)
          >>> rh = cluster half mass radius (pc)
          >>> trh = relaxation time (Myrs)
          
    Output >>> tint = interaction timescale (Myrs)
    '''
    
    # Convert rh to AU
    rh*=pc2AU
    
    binary = (M1*M2)/a
    cluster = rh/(Mcl**2)
    
    return 5 * binary * cluster * trh

def calcMergeFromInteractions(m1, m2, semi, tint, N):
    '''
    Calculate the affect of an interaction and see if it would 
    lead to a binary that merges. For each binary assume that the 
    binding energy increases by 40% and the eccentricity is drawn 
    from a thermal distribution averaged over 10 times
    
    Input >>> m1, m2 = primary and secondary binary mass (Msol)
          >>> Semi = semi-major axis (rsol)
          >>> tint = time for an interaction (Myrs)
          >>> N = Number of interactions to average over
          
    Output >>> avgtdelay = coalescence time averaged over 10 interactions (Myrs)
    '''
    
    # Number of binaries
    num=m1.size
    
    # Assume hard encounter increases binding energy by 40%
    new_a = semi/1.4
    
    # Empty array to store all of the Tdelays
    delayTime_all = np.zeros(num)
    
    for i in range(N):
        # Draw an eccentricity from a thermal distribution for every binary
        esq = np.random.uniform(0,1,num)
        e = np.sqrt(esq)
        
        # Find the merger time for each of the binaries in Myrs and + interaction time
        tmerge = tdelay(ai=new_a, ei=e, m1=m1, m2=m2)/1e6
        delayTime = tmerge+tint
        
        # Append to the array we have
        delayTime_all = np.vstack((delayTime_all, delayTime))
    
    # for each binary average the tdelays
    delayTime_all = delayTime_all.T
    avgtdelay = np.mean(delayTime_all, axis=1)
    
    return avgtdelay
    

In [4]:
# Defining constants
G = 1.908e5 # R_sol*(M_sol)^-1*km^2*s^-2 
pc2AU = 206265 # Pc -> AU
Rsol2AU = 0.00465047 # Rsol -> AU
pcMyr2kms = 1.023 # Pc/Myr -> km/s

In [103]:
# Selecting directory
dataDir = find_dir()

Possible Directories:

0 : COMPAS_Output_1%solar_metallicity
1 : COMPAS_Output_10%solar_metallicity
2 : COMPAS_Output_PeTar_M100000
3 : COMPAS_Output_SanaDist_1%metal
4 : COMPAS_Output_SanaDist_1%metal_2
5 : COMPAS_Output_SanaDist_10%metal
6 : COMPAS_Output_SanaDist_10%metal_2
7 : COMPAS_Output_SanaDist_solmetal
8 : COMPAS_Output_SanaDist_solmetal_2
9 : COMPAS_Output_solar_metallicity



What directory is the data stored in?   4


In [104]:
# Load in the double compact objects as well as the system parameters
DCO = pd.read_csv(os.path.join(dataDir, 'BSE_Double_Compact_Objects.csv'), skiprows=2)
SP = pd.read_csv(os.path.join(dataDir, 'BSE_System_Parameters.csv'), skiprows=2)
SN = pd.read_csv(os.path.join(dataDir, 'BSE_Supernovae.csv'), skiprows=2)


# Find the equilibrated at birth and remove them from the DCOs
EAB = SP.loc[SP['Equilibrated_At_Birth']==1] 
DCO.drop(DCO.loc[DCO['    SEED    '].isin(EAB['    SEED    '])].index, inplace=True)
SN.drop(SN.loc[SN['    SEED    '].isin(EAB['    SEED    '])].index, inplace=True)

# Remove the invalid values
invalidVals = SN.loc[(SN['SystemicSpeed '] == '          -nan')|(SN['SystemicSpeed '] == '          -nan')|(SN['SystemicSpeed '] == '          -nan')]
if len(invalidVals)>0:
    print('{} systems dropped'.format(len(invalidVals)))
    SN.drop(invalidVals.index, inplace=True)

SN = SN.astype({'SystemicSpeed ':'float64', 
                'ComponentSpeed(SN)':'float64', 
                'ComponentSpeed(CP)':'float64',
                'SemiMajorAxis ':'float64'})



# Specifically grab the BBHs
BBHMaster = DCO.loc[(DCO['Stellar_Type(1)']==14)&(DCO['Stellar_Type(2)']==14)].copy()
BBHMaster.reset_index(inplace=True, drop=True)

In [105]:
'''
Here we find all of the possible BH systems so that we can 
later find which have been retained.
'''

# Index for both SNs , only first and only last
SNDupIndex = SN.duplicated(subset='    SEED    ', keep=False)

SN1st = SN.loc[SN.duplicated(subset='    SEED    ', keep='last')]
SN2nd = SN.loc[SN.duplicated(subset='    SEED    ', keep='first')]

SN1st.reset_index(drop=True, inplace=True)
SN2nd.reset_index(drop=True, inplace=True)

# Two SNs
SNDup = SN.loc[SNDupIndex]
SNDup.reset_index(inplace=True, drop=True)

# Single SN
SNSing = SN.loc[~SNDupIndex]
SNSing.reset_index(inplace=True, drop=True)

# BH other star unbound and bound
BHSingUnbound = SNSing.loc[(SNSing['Stellar_Type(SN)']==14)&(SNSing['Unbound']==1)]
BHSingBound = SNSing.loc[(SNSing['Stellar_Type(SN)']==14)&(SNSing['Unbound']==0)]

BHSingUnbound.reset_index(inplace=True, drop=True)
BHSingBound.reset_index(inplace=True, drop=True)

# BBHs that remain bound
BBHBound = SN.loc[(SN['Stellar_Type(SN)']==14)&(SN['Stellar_Type(CP)']==14)&(SN['Unbound']==0)]
BBHBound.reset_index(inplace=True, drop=True)

# BH other SN
BHElse = SN2nd.loc[((SN2nd['Stellar_Type(SN)']==14)&(SN2nd['Stellar_Type(CP)']!=14))|((SN2nd['Stellar_Type(CP)']==14)&(SN2nd['Stellar_Type(SN)']!=14))]
BHElse.reset_index(drop=True, inplace=True)

# BBHs that are not bound
BBHUnbound = SN.loc[(SN['Stellar_Type(SN)']==14)&(SN['Stellar_Type(CP)']==14)&(SN['Unbound']==1)]
BBHUnbound.reset_index(inplace=True, drop=True)


In [106]:
SN.keys()

Index(['    SEED    ', 'Drawn_Kick_Magnitude(SN)',
       'Applied_Kick_Magnitude(SN)', 'Fallback_Fraction(SN)',
       'Orb_Velocity<SN', 'Kick_Magnitude(uK)', 'True_Anomaly(psi)(SN)',
       'SN_Kick_Theta(SN)', 'SN_Kick_Phi(SN)', 'SN_Type(SN)',
       'Eccentricity<SN', ' Eccentricity ', 'SemiMajorAxis<SN',
       'SemiMajorAxis ', '      Time      ', 'Supernova_State', 'Unbound',
       'Stellar_Type(CP)', 'Stellar_Type(SN)', 'Stellar_Type_Prev(SN)',
       '   Mass(CP)   ', 'Mass_Total@CO(SN)', 'Mass_Core@CO(SN)',
       'Mass_CO_Core@CO(SN)', 'Mass_He_Core@CO(SN)', '   Mass(SN)   ',
       'Experienced_RLOF(SN)', 'MT_Donor_Hist(SN)', 'ComponentSpeed(SN)',
       'ComponentSpeed(CP)', 'SystemicSpeed ', 'Is_Hydrogen_Poor(SN)'],
      dtype='object')

In [139]:
'''
Here I define either a specific mass or a specific escape velocity 
'''
Mcl = 100 # 3 values of cluster mass (Msol) for open, globular and nuclear clusters densities
rh = 2.154 # pc
rhoh = (Mcl/2)/(4/3*np.pi*rh**3)

vesc = 280 # km/s
sigma = vesc/4.77 # km/s

# Assumed perturber mass
m3=20 # Msol 

perturberMass = np.array([])

# Make a copy of the data
BBH_simple = BBHMaster.copy()

# Reset the counters
retainedSingle=0
retainedBound=0

# This last part will calculate the fraction of retained binaries (containing at least one BH) with the cluster 
# Those systems that form a BH but the second star never goes SN
retainedSingle+=len(BHSingUnbound.loc[BHSingUnbound['ComponentSpeed(SN)']<vesc])
retainedBound+=len(BHSingBound.loc[BHSingBound['ComponentSpeed(SN)']<vesc])

# seeds retained from first SN
retainedInFirst = SN1st.loc[(SN1st['SystemicSpeed ']<vesc)&(SN1st['Unbound']==0)]
retainedSN = SN1st.loc[(SN1st['ComponentSpeed(SN)']<vesc)&(SN1st['Unbound']==1)]
retainedCP = SN1st.loc[(SN1st['ComponentSpeed(CP)']<vesc)&(SN1st['Unbound']==1)]

# BBHbound retained after second
index = BBHBound['    SEED    '].isin(retainedInFirst['    SEED    '])
BBHRetain = BBHBound.loc[(BBHBound['SystemicSpeed ']<vesc)&(index)]
retainedBound+=2*len(BBHRetain)

# BBHUnbound on second
index = BBHUnbound['    SEED    '].isin(retainedInFirst['    SEED    '])
retainedSingle+=len(BBHUnbound.loc[(index)&(BBHUnbound['ComponentSpeed(SN)']<vesc)])
retainedSingle+=len(BBHUnbound.loc[(index)&(BBHUnbound['ComponentSpeed(CP)']<vesc)])

# Add single mass to array
perturberMass = np.append(perturberMass, BBHUnbound['   Mass(SN)   '].loc[(index)&(BBHUnbound['ComponentSpeed(SN)']<vesc)].values)
perturberMass = np.append(perturberMass, BBHUnbound['   Mass(CP)   '].loc[(index)&(BBHUnbound['ComponentSpeed(CP)']<vesc)].values)

# BHElse on Second
index = BHElse['    SEED    '].isin(retainedInFirst['    SEED    '])
retainedBound+=len(BHElse.loc[(index)&(BHElse['SystemicSpeed ']<vesc)&(BHElse['Unbound']==0)])
retainedSingle+=len(BHElse.loc[(index)&(BHElse['ComponentSpeed(SN)']<vesc)&(BHElse['Stellar_Type(SN)']==14)&(BHElse['Unbound']==1)])
retainedSingle+=len(BHElse.loc[(index)&(BHElse['ComponentSpeed(CP)']<vesc)&(BHElse['Stellar_Type(CP)']==14)&(BHElse['Unbound']==1)])

# Add single masses to array
perturberMass = np.append(perturberMass, BHElse['   Mass(SN)   '].loc[(index)&(BHElse['ComponentSpeed(SN)']<vesc)&(BHElse['Stellar_Type(SN)']==14)&(BHElse['Unbound']==1)].values)
perturberMass = np.append(perturberMass, BHElse['   Mass(CP)   '].loc[(index)&(BHElse['ComponentSpeed(CP)']<vesc)&(BHElse['Stellar_Type(CP)']==14)&(BHElse['Unbound']==1)].values)

# retainedUnbound on first
retainedSingle+=len(retainedSN.loc[retainedSN['Stellar_Type(SN)']==14])
index = SN2nd['    SEED    '].isin(retainedCP['    SEED    '])
retainedSingle+=len(SN2nd.loc[(index)&(SN2nd['ComponentSpeed(SN)']<vesc)&(SN2nd['Stellar_Type(SN)']==14)])

# Add single masses to array
perturberMass = np.append(perturberMass, retainedSN['   Mass(SN)   '].loc[retainedSN['Stellar_Type(SN)']==14].values)
perturberMass = np.append(perturberMass, SN2nd['   Mass(SN)   '].loc[(index)&(SN2nd['ComponentSpeed(SN)']<vesc)&(SN2nd['Stellar_Type(SN)']==14)].values)


# Append to the fractional arrays
totalretained=retainedSingle+retainedBound

# Find hard binaries
mu = (BBHRetain['   Mass(SN)   ']*BBHRetain['   Mass(CP)   '])/(BBHRetain['   Mass(SN)   ']+BBHRetain['   Mass(CP)   '])
ah = (G*mu)/(sigma**2)
hardBBH = BBHRetain.loc[BBHRetain['SemiMajorAxis ']<=ah]

# Make a distribution of single BH masses 
values, bins = np.histogram(perturberMass, bins = range(0, round(max(perturberMass)), 1), density = True)
bin_mid = np.array([(bins[j+1]+bins[j])/2 for j in range(len(bins)-1)])

m_perturb = choices(bin_mid, weights=values, k=len(hardBBH))

# Calculate the recoil kick from a single interaction assuming 0.2 energy transfer
kick_vel_sq = 0.2 * G/hardBBH['SemiMajorAxis '] * (hardBBH['   Mass(SN)   '] * hardBBH['   Mass(CP)   '])/(hardBBH['   Mass(SN)   '] + hardBBH['   Mass(CP)   '] + m_perturb)

# Find hard binaries removed after single interaction
removed1st = hardBBH.loc[kick_vel_sq >= vesc**2]

In [140]:
(len(hardBBH) - len(removed1st))/len(hardBBH) * 100

99.73092446665386

In [61]:
len(removed1st)

175